In [1]:
import pandas as pd
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
from pandas import DataFrame 
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma
from konlpy.tag import Okt
from collections import Counter
from konlpy.tag import Mecab

from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
from gensim.models.coherencemodel import CoherenceModel
from IPython.display import clear_output
import logging
from ast import literal_eval
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
class TopicModeling:
    def __init__(self,path=None,movie_data=None):
        self.stopwords=[]
        file = open(f"koreanStopwords2.txt", "r")
        while True:
            line = file.readline()
            if not line:
                break
            self.stopwords.append(line.strip())
        file.close()
        self.original_path = path
        self.kkma = Kkma()
        self.mecab=Mecab()
        if path != None:
            self.mbti = pd.read_csv(self.original_path,index_col=0,header=0)
        else:
            self.movie = movie_data

    def clean_text(self,text):
        text=re.sub(r"[^A-Za-z0-9가-힣.]|[xa0]"," ",text)
        text =' '.join(text.split())
        text = text.lstrip()
        return text.lower()
    
    def sentence_split(self, text):
        sentence = sum([],(text.split('.')))
        sentence = list(filter(None, sentence))
        return [s.strip() for s in sentence]
    
    def get_nouns(self,tokenizer, sentence):
        tagged = tokenizer.pos(sentence)
        nouns = [s for s, t in tagged if t in ['NNG', 'NNP', 'VA', 'XR'] and len(s) >1]
        return nouns
    
    def get_nouns(self,tokenizer, sentence):
        tagged = tokenizer.pos(sentence)
        nouns = [s for s, t in tagged if t in ['NNG', 'NNP', 'VA', 'XR'] and len(s) >1]
        nouns = [x for x in nouns if x not in self.stopwords]
        return nouns
    
    def keyword_ext(self,mbti=None,movie_title=None,topic=5,passes=10,iterations=10):
        keyword = []
        if mbti != None:
            print('hi')
            mbti_ = literal_eval(self.mbti.loc[mbti]['contents'])
            for key in mbti_:
                keyword.append(self.get_nouns(self.mecab,key))

        else:
            try:
                movie_ = literal_eval(self.movie)
                if len(movie_)==1:
                    movie_ = self.sentence_split(movie_[0])
                    if len(movie_)==1:
                        return ''
                for key in movie_:
                    keyword.append(self.get_nouns(self.mecab,key))
            except:
                return ''
        dictionary = corpora.Dictionary(keyword)
        dictionary.filter_extremes(no_below=2, no_above=0.3)
        corpus = [dictionary.doc2bow(text) for text in keyword]
        num_topics = topic
        chunksize = 1000
        passes = passes
        iterations = iterations
        eval_every = 0
        try:
            temp = dictionary[0]
        except:
            return ''

            
            
        id2word = dictionary.id2token

        model = LdaModel(
            corpus=corpus,
            id2word=id2word,
            chunksize=chunksize,
            alpha='auto',
            eta='auto',
            iterations=iterations,
            num_topics=num_topics,
            passes=passes,
            eval_every=eval_every
        )
        return  model.top_topics(corpus)
    
    def clean_topic(self,topics):
        topic_dict={}
        for i in range(len(topics)):
            if topics[i][1]>-10.0 and topics[i][1]<-1.5:
                continue
            for key,value in dict(topics[i][0]).items():
                try:
                    topic_dict[value] += key
                except KeyError:
                    topic_dict[value] = key
        return topic_dict
    

In [ ]:

mbti_list = ['ISTJ','ISFJ','INFJ','INTJ',
                        'ISTP','ISFP','INFP','INTP',
                        'ESTP','ESFP','ENFP','ENTP',
                        'ESTJ','ESFJ','ENFJ','ENTJ']
model = None
df = pd.DataFrame()
for mbti in mbti_list:
    topic_list = []
    model = TopicModeling(path='./mbti_final_data.csv')
    topic = model.keyword_ext(mbti)
    topic_list += topic

    final_topic = model.clean_topic(topic_list)
    topic_df = pd.DataFrame(final_topic.items(),columns=[[f'{mbti}_keyword',f'{mbti}_weight'],['keyword','weight']])
    df = pd.concat([df,topic_df], axis=1)


In [6]:
movie = pd.read_csv('./total_review.csv',index_col=0,header=0)
movie.replace('[]',np.nan,inplace=True)

In [ ]:
model = None
count = 0
col_names = ['Watcha_Contents','Naver_Contents','Tistory_Contents']
not_found = {}
for col in col_names:
        df = pd.DataFrame()
        count=0
        for title,n_id in list(zip(movie.index,movie.nfid)):
            clear_output()
            print(f"진행상황 = {col}_{count}번째")
            topic_list = []
            model = TopicModeling(movie_data=movie[movie.nfid==n_id][col][0])

            topic = model.keyword_ext(movie_title=title,topic=10,passes=50,iterations=100)

            topic_list += topic
            final_topic = model.clean_topic(topic_list)
            topic_df = pd.DataFrame(final_topic.items(),columns=[[f'{title}_{n_id}_keyword',f'{title}_{n_id}_weight'],['keyword','weight']])
            df = pd.concat([df,topic_df], axis=1)
            count+=1
        df.to_csv(f'{col}_topic3.csv',encoding='utf-8')


In [2]:
import pandas as pd
pd.read_csv('./total_review.csv')

,title,nfid,Watcha_Contents,Naver_Title,Naver_Contents,Tistory_Title,Tistory_Contents
0,시그널,80987077,"['돈 주고 표 끊고 봐야할거 같은 드라마', '앞으로 극의전개가 매우궁금하다 조금...",[],[],[],[]
1,조이,81046255,"['여자가 사업을 좀 하겠다는데 짜식들이 방해를 하구 말이야', '구심력을 상실한 ...",[],[],[],[]
2,히트,70020509,['신선하다. 여자배우들이 더 많이 영화에서 중심이 되면 좋겠다. 남자투톱영화 지겨...,[],[],[],[]
3,아가씨,80113804,"['스포일러가 있어요!! 보기', '스포일러가 있어요!! 보기', '스포일러가 있어...",[],[],[],[]
4,도가니,70241118,"['공론화했다는 것만으로도 가치있는 영화', '별4개를 매기면 뜨는 재밌어요! 는 ...",[],[],[],[]
...,...,...,...,...,...,...,...
1432,퀸메이커,81503026,['그 뻔하고 올드한 걸 김희애 문소리는 왜 이제서야 찍을 수 있었는지가 중요함.....,"['넷플릭스 퀸메이커 후기 결말 전형적인 드라마 ', '퀸메이커 넷플릭스 서울시장하...",['넷플릭스 퀸메이커 후기 결말 전형적인 드라마 넷플릭스 오리지널 한국 드라마 추천...,"['[드라마리뷰] 퀸메이커 등장인물, 시즌2, 결말, 몇부작, 은채령 배우?', '...","[""*주의 강력한 스포가 포함되어있습니다.퀸메이커 결말 시즌2요즘 영화나 드라마에서..."
1433,닥터 차정숙,81682438,['엄정화 개멋있다.엄마들은 정말 대단한 존재이다.엄마도 나처럼 꿈이 있다는걸 잊으...,"[' 번역탐구 넷플릭스 드라마 닥터 차정숙 3화 영어 자막 ', '닥터 차정숙 홀릭...",[' 번역탐구 넷플릭스 드라마 닥터 차정숙 3화 영어 자막 업템포글로벌 번역 닥터 ...,"['JTBC 드라마 <닥터 차정숙> 줄거리, 등장인물 소개, 리뷰', '닥터 차정숙...",['의대생이었던 정숙은 혼전임신으로 인해 의대 졸업 후 20년 동안 평범한 가정주부...
1434,끝나지 않는 세 번째 데이트,81456532,['모든 데이트는 운명적이다. 그러나 어떤 데이트는 다른 데이트보다 더 운명적이다....,"['끝나지 않는 세 번째 데이트 넷플릭스 짧은 요약 스포 있음 ', '끝나지 않는 ...",['끝나지 않는 세 번째 데이트 넷플릭스 짧은 요약 스포 있음 넷플릭스 영화 다큐시...,[],[]
1435,나쁜엄마,81669775,"['와 개잘만들었다', '모든 캐릭터를 절벽끝에 세워두고시작했는데 생각보다 절벽이 ...","['나쁜엄마 10화 ', '나쁜엄마 라미란 이도현의 복수 빅피쳐를 알게 된다 9회 ...",['나쁜엄마 10화 안녕하세요 여러분제가 제일 좋아하는 드라마중에 하나나쁜엄마 본방...,"['넷플릭스 볼만한 드라마 [나쁜엄마] 줄거리, 결말 리뷰', '나쁜엄마 14회 마...","[""등장인물~더 글로리 이후에 이도현배우가 출연한다고 나와서 더 보고 싶기도 했던 ..."
